# Project 2

In [1]:
import torch
import math
torch.set_grad_enabled(False)

In [2]:
class Module(object):
    def __init__(self):
        self.parameters = []
        
    def forward(self, *input):
        raise NotImplementedError
        
    def backward(self, *gradwrtoutput):
        raise NotImplementedError
        
    def param(self):
        return self.parameters

In [3]:
class Linear(Module):
    def __init__(self, in_features, out_features, bias=False):
        init_range = math.sqrt(in_features)
        self.weights = torch.Tensor(in_features, out_features).uniform_(-init_range, init_range)
        self.grad_w = torch.zeros((in_features, out_features))
        self.parameters = [(self.weights, self.grad_w)]
        if bias:
            self.bias = torch.Tensor(out_features).uniform_(-init_range, init_range)
            self.grad_b = torch.zeros(out_features)
            self.parameters.append((self.bias, self.grad_b))
        else:
            self.bias = None
        
    def forward(self, input_):
        self.input = input_
        if self.bias != None:
            return torch.addmm(self.bias, input_, self.weights)
        else:
            return input_.matmul(self.weights)
        
    def backward(self, grad_output):
        self.grad_w += self.input.t().matmul(grad_output)
        grad_input = grad_output.matmul(self.weights.t())
        if self.bias != None:
            self.grad_b += grad_output.sum(dim=0)
        return grad_input

In [4]:
class ReLU(Module):
    
    def forward(self, input_):
        self.input = input_
        return torch.relu(input_)
    
    def backward(self, grad_output):
        return torch.mul((self.input > 0).int(), grad_output)

In [5]:
class MSELoss(Module):
    def __init__(self):
        pass
    
    def forward(self, input_, target):
        if target.dim() == 1:
            target = target.view(target.size(0), 1)
        return (input_ - target).pow(2).sum()
    
    def backward(self, input_, target):
        if target.dim() == 1:
            target = target.view(target.size(0), 1)
        
        return (input_ - target).mul(2)

In [6]:
class Sequential(Module):
    def __init__(self, *args):
        self.parameters = []
        for module in args:
            self.parameters += module.parameters
# TODO !!!

In [7]:
def generate_set(size):
    input_ = torch.Tensor(size, 2).uniform_(0, 1)
    target = input_.sub(0.5).pow(2).sum(axis=1).sub(1 / math.pi ** 2).sign().add(1).div(2)
    return input_, target

train_input, train_target = generate_set(200)
test_input, test_target = generate_set(200)

In [8]:
class Model:
    def __init__(self, layers):
        self.layers = layers
        self.parameters = []
        for layer in layers:
            self.parameters += layer.parameters
    
    def zero_grad(self):
        for w, dw in self.parameters:
            dw.zero_()
    
    def forward(self, input_):
        x = input_
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def backward(self, loss_grad):
        y = loss_grad
        for layer in reversed(self.layers):
            y = layer.backward(y)
        return

In [9]:
class SGD:
    def __init__(self, parameters, learning_rate):
        self.parameters = parameters
        self.learning_rate = learning_rate
        
    def step(self):
        for w, dw in self.parameters:
            w -= self.learning_rate * dw

In [10]:
layers = [Linear(2, 10), ReLU(), Linear(10, 5), ReLU(), Linear(5, 1)]
model = Model(layers)
sgd = SGD(model.parameters, 0.0001)
loss = MSELoss()

In [11]:
for epoch in range(25):
    print(epoch)
    model.zero_grad()
    output = model.forward(train_input)
    print(loss.forward(output, train_target).item())
    loss_grad = loss.backward(output, train_target)
    model.backward(loss_grad)
    sgd.step()

0
156.3612518310547
1
143.93032836914062
2
134.49656677246094
3
127.0963134765625
4
121.004638671875
5
115.86997985839844
6
111.46519470214844
7
107.68602752685547
8
104.46123504638672
9
101.69342041015625
10
99.25413513183594
11
97.1548843383789
12
95.36774444580078
13
93.86734771728516
14
92.60344696044922
15
91.55255126953125
16
90.67800903320312
17
89.94464874267578
18
89.31330871582031
19
88.79692840576172
20
88.37225341796875
21
88.02761840820312
22
87.74763488769531
23
87.51737976074219
24
87.32195281982422
